In [1]:
import pandas as pd
import numpy as np
import re
import math
import pickle

from patsy import dmatrices
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from gensim import corpora, models, similarities


In [2]:
df = pd.read_csv('./saved_models/merged_df_bills_topics.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,0,1,2,3,4,5,6,7,...,92,93,94,95,96,97,98,99,bill_id,bill_status
0,0,0,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.972691,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,200920101AB9,0
1,1,14592,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.972684,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,200920101AB9,0
2,2,1,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.972738,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,200920101AB13,0
3,3,14593,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.972664,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,200920101AB13,0
4,4,2,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.972687,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,200920101AB11,0


In [3]:
#remove wierd index columns
df2 = df.iloc[:, 2:-2] 
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.972691,-0.0001,-0.0001,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001
1,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.972684,-0.0001,-0.0001,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001
2,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.972738,-0.0001,-0.0001,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001
3,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.972664,-0.0001,-0.0001,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001
4,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,0.972687,-0.0001,-0.0001,-0.0001,...,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001,-0.0001


In [4]:
# get the right rows for the models
x = df2.values
xScaled = StandardScaler().fit_transform(x)

y = df['bill_status'].values

In [5]:
print 'starting starting'
X_train, X_test, y_train, y_test = train_test_split(xScaled, y, 
    test_size=0.33, random_state=42)


starting starting


In [ ]:
param_grid = [{'lr__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}]

pipe_lr = Pipeline(steps=[ ('lr', LogisticRegression(random_state=1046)) ])

#added statifiedKfolds for cv to decrease overfitting
#cvs = StratifiedKFold(y_train, n_folds = 5, shuffle=True)

gs = GridSearchCV(estimator=pipe_lr, param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=5,n_jobs=-1)
gs = gs.fit(X_train, y_train)

In [ ]:
print('Grid Search Best Score: %.4f' % gs.best_score_)
print('Grid Search Best Parameter for C: ')
print gs.best_params_

for params, mean_score, scores in gs.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r"
          % (mean_score, scores.std() / 2, params))

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
scores = cross_val_score(gs, X, y, cv=3, scoring=metric)
print('cross_val_score', metric, scores.mean(), scores.std())